## JSON Agent

In [1]:
from dotenv import load_dotenv
import os
import json
import openai
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_community.document_loaders import UnstructuredFileLoader

load_dotenv()


True

In [2]:
def extract_text_from_file(file_path):
    text_data = []
    try:
        loader = UnstructuredFileLoader(file_path)
        docs = loader.load()
        text_data.append("\n\n".join([doc.page_content for doc in docs]))
    except Exception as e:
        print(f"⚠️ Ошибка при обработке {file_path}: {e}")

    return "\n\n".join(text_data)

def convert_to_json(text):
    prompt = f"""
        Ты получишь текстовые данные с описанием товаров. Они могут быть как табличными так и текст с описанием товаров.  
        Верни два списка со следующими структурами. 
        1. Общая характеристика заявки со струткурой:
        [
        {{
            "номер заявки": int,         // используй UUID v4 для генерации уникального номера заявки
            "дата": datetime,  // дата прихода заявки
            "товары": [int]        // уникальные id для каждого товара в этой заявке 
        }}
        ]
        2.
        [
        {{
            "ID": int,         // id-товара из первого json файла
            "описание": "string" // описание товара, если нет оставь пустым 
            "наименование": "string",  // наименование товара
            "размеры": ["string/int"],      // Список доступных размеров (если нет, укажи []), размеры могут иметь следующий вид: l70-176;48-50;S/M/L
            "кол-во": ["int"]         // Количество товаров для каждого размера или общее количество (если размеров нет, укажи [общее число]), количество часто это число перед шт. : 5 шт.
        }}
        ]
        Правила обработки данных:
        Извлекай только нужные данные: игнорируй любые другие колонки и текст.
        "размеры" – если размеры указаны, представь их в виде списка ["S", "M", "L"] или ["44", "56"], если их нет — укажи ["one-size"].
        "кол-во" – Если есть размеры, указывай количество товаров для каждого размера в том же порядке, что и "размеры". 
        Если размеров нет, но указано общее количество, записывай его как список с одним числом ["количество"]. 
        
        Важно:
        Если есть таблицы: анализируй заголовки таблиц и извлекай данные корректно.
        Соблюдай указанную структуру без лишний слов. Ответ должен содержать два массива через запятую.

        Вот данные:
        {text}
    """
    
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    response = llm([HumanMessage(content=prompt)])
    return response.content.strip("```json").strip("```")

def process_files(dir):
    
    for file_name in os.listdir(dir):
        file_path = os.path.join(dir, file_name)
        text_data = extract_text_from_file(file_path)
        json_data = convert_to_json(text_data)
    
        #return json_data
        output_file = os.path.splitext(f'json_price/{file_name}')[0]+'.json'

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(json.loads(json_data), f, indent=4, ensure_ascii=False)
        print(f"✅ Данные сохранены в {output_file}!")

In [10]:
text_data = extract_text_from_file("exmp_data/ЗАЯВКА НА СПЕЦОДЕЖДУ (1).pdf")
json_data = convert_to_json(text_data)


In [11]:
print(json_data)


[
    {
        "номер заявки": "f47ac10b-58cc-4372-a567-0e02b2c3d479",
        "дата": "2024-02-22T00:00:00",
        "товары": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    }
],
[
    {
        "ID": 1,
        "описание": "",
        "наименование": "Костюм (6прокн)",
        "размеры": ["44", "46", "48", "50", "52", "54", "56", "58", "60", "62", "64", "66", "68", "70"],
        "кол-во": ["5", "30", "15", "5", "90", "70", "5", "30", "5", "10", "5", "2", "1"]
    },
    {
        "ID": 2,
        "описание": "",
        "наименование": "Костюм (6прокн)",
        "размеры": ["40", "42"],
        "кол-во": ["2", "3"]
    },
    {
        "ID": 3,
        "описание": "",
        "наименование": "Костюм (6прокн)",
        "размеры": ["44", "46"],
        "кол-во": ["30", "10"]
    },
    {
        "ID": 4,
        "описание": "",
        "наименование": "Костюм (6прокн)",
        "размеры": ["48", "50"],
        "кол-во": ["25", "30"]
    },
    {
        "ID": 5,
        "описание": "",
       

In [51]:
process_files("датасет_прайс")

✅ Данные сохранены в json_price/Прайc_на_ОБУВЬ_ЛЕТО_сентябрь_2024.json!
✅ Данные сохранены в json_price/Прайc_на_перчатки_сентябрь_2024.json!
✅ Данные сохранены в json_price/Прайс лист_ЗИМА_2024-2025.json!
✅ Данные сохранены в json_price/Прайс лист_ЗИМА_перекупные позиции.json!
✅ Данные сохранены в json_price/Прайс лист_СИЗ_2024-2025.json!
✅ Данные сохранены в json_price/Прайс_лист_Швецъ_ЛЕТО_2024.json!


## vecotr db

In [4]:
#from langchain_ollama import OllamaEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

from uuid import uuid4
import json
from langchain.schema import Document
import os

In [2]:
import re
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = text.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmatized_words)

In [5]:
docs = []

for file_name in os.listdir("json_price"):
    file_path = "json_price/" + file_name
    with open(file_path, "r", encoding="utf-8") as f:
        products = json.load(f)


    for product in products:
        name = preprocess_text(product["name"])
        description = preprocess_text(product["description"])
        metadata = {key: str(product[key]) for key in product if key not in ["name", "description"]}
        metadata["currency"] = "RUB"

        # Create a LangChain Document object
        doc = Document(page_content=f"{name}. {description}", metadata=metadata)
        docs.append(doc)

In [6]:
from langchain_chroma import Chroma


#model_name = "DeepPavlov/rubert-base-cased"
#embedding_model = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-base")
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large")

vector_store = Chroma(
    collection_name="products",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db_price",  # Where to save data locally, remove if not necessary
)

In [7]:
uuids = [str(uuid4()) for _ in range(len(docs))]

In [ ]:
vector_store.add_documents(documents=docs, ids = uuids)

## retriever

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(embedding_function=embedding_model, collection_name = "products", persist_directory="./chroma_langchain_db_price")

In [8]:
retriever = vector_store.as_retriever(search_type = "mmr")

In [9]:
retriever.invoke("Кепки")

[Document(id='04ab094b-5110-4142-b88c-10fad67ba472', metadata={'currency': 'RUB', 'price': '290', 'sizes': "['one-size']", 'stock': '[1]'}, page_content='кепка- бейсболка бежевый с чёрный козырёк. '),
 Document(id='19b9db68-6f2c-4797-8cf6-d13957db10ca', metadata={'currency': 'RUB', 'price': '290', 'sizes': "['one-size']", 'stock': '[1]'}, page_content='шапка трикотажный. '),
 Document(id='b785c72f-ae68-4e0f-a9cc-9f2cca79bbd8', metadata={'currency': 'RUB', 'price': '280', 'sizes': "['XXL']", 'stock': '[1]'}, page_content='кепи охранник кмф голубой. '),
 Document(id='16a1ad8e-abec-413e-b911-f8d52831ea54', metadata={'currency': 'RUB', 'price': '3500', 'sizes': "['41', '43', '45']", 'stock': '[5, 1, 3]'}, page_content='ботинок "корвет" кпк. ')]

## rag

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-4o-mini")

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
Ты – эксперт по подбору и производству спецодежды. 
Ты получаешь на вход следующие JSON-структуры, которые надо дополнить недостоющими параметрами(статусы, аналоги):
{question}
1. **Общая характеристика заявки**:
    ```json
    [
        {{
            "номер заявки": "UUID",    // Генерируй уникальный UUID v4
            "дата": "datetime",       // Дата поступления заявки
            "товары": [int]           // Список уникальных ID товаров в этой заявке
            "статус": string            // Определяется по правилам ниже
        }}
    ]
    ```

2. **Детализация каждого товара в заявке**:
    ```json
    [
        {{
            "ID": int,                    // ID товара из первого JSON
            "описание": "string",         // Описание товара (если нет, оставь пустым)
            "наименование": "string",     // Наименование товара
            "размеры": ["string/int"],    // Доступные размеры (если нет, укажи [])
            "кол-во": ["int"],            // Количество по размерам или общее (если размеров нет, укажи [общее число])
            "статус": "string",           // Определяется по правилам ниже
            "аналоги": ["string"]         // Перечисли найденные аналоги (если нет аналогов, укажи [])
        }}
    ]
    ```

### Логика определения **"статуса"** для товаров:
- `"ШИТЬ"` → Если товар является спецодеждой.
- `"КОМПЛЕМЕНТАРНОЕ"` → Если товар не является одеждой (например, ботинки, кепки, очки).
- `"НЕРЕЛЕВАНТНО"` → Если товар не относится к спецодежде и СИЗ.

### Заполнение поля **"аналоги"**:
- Подбирай аналоги товара из `{retrieved_products}`.
- Если есть подходящие аналоги, укажи их наименования в массиве.
- Если аналогов нет, укажи `[]`.

### Логика определения **"статуса"** заявки:
- `"ШИТЬ"` → Если хотя бы один товар в JSON 2 имеет статус `"ШИТЬ"`.
- `"АНАЛОГИ"` → Если у всех товаров со статусом `"ШИТЬ"` есть аналоги.
- `"НЕРЕЛЕВАНТНО"` → Если хотя бы один товар имеет статус `"НЕРЕЛЕВАНТНО"`.

### Формат ответа:
Ответ должен содержать **два массива**, разделенных словом `next`, не пиши ```json```:


[ ... ]  // JSON 1
next
[ ... ]  // JSON 2
"""

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(input_variables=["retrieved_products", "question"], template=template)

llm_chain = prompt | llm | StrOutputParser()

In [25]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

rag_chain = {"retrieved_products":retriever , "question": RunnablePassthrough()} | llm_chain

In [26]:
request, products = rag_chain.invoke(json_data).split("next")

In [30]:
df_zayavki = pd.DataFrame(json.loads(request))
df_tovary = pd.DataFrame(json.loads(products))

In [31]:
df_zayavki

,номер заявки,дата,товары,статус
0,b2c7a74e-9449-4f2c-9b2f-d39a3e14b06f,2023-10-01T00:00:00,"[1, 2, 3]",ШИТЬ


In [32]:
df_tovary

,ID,описание,наименование,размеры,кол-во,статус,аналоги
0,1,,Кепки,[one-size],[100],КОМПЛЕМЕНТАРНОЕ,[]
1,2,,Футболки с логотипом на груди,"[48-50, 52-54, 60-62]","[2, 1, 5]",ШИТЬ,[]
2,3,,Футболки Поло с двумя логотипами,"[48-50, 52-54, 56-58]","[2, 3, 2]",ШИТЬ,[]
